In [7]:
import cv2
import time
import FaceNormalizationUtils as faceutils
import FaceDetectors

# Normalization instance
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Load the mask image with a transparent background
mask = cv2.imread('images/mascara-batman.png', cv2.IMREAD_UNCHANGED)

# Webcam connection
cap = cv2.VideoCapture(0)

# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

# Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

# Set camera resolution
cap.set(3, 640)
cap.set(4, 480)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    # Split color channels
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            [lex, ley, rex, rey] = eyes
            if lex > -1:
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, (int(lex), int(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, (int(rex), int(rey)), 4, (0, 255, 0), -1)

                # Superimpose the mask on the face
                mask_x = x
                mask_y = y
                mask_w = w
                mask_h = h
                mask2 = cv2.resize(mask, (mask_w, mask_h))

                for c in range(0, 3):
                    frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * (1 - mask[:, :, 3] / 255.0) + mask[:, :, c] * (mask[:, :, 3] / 255.0)

    if debug:
        print("Processing time: {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)

    # Press 'Esc' to exit
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windows and release the camera
cap.release()
cv2.destroyAllWindows()


Camera 0


ValueError: operands could not be broadcast together with shapes (168,168) (1080,810) 

: 